In [1]:
import os  
# set env LOG_LEVEL=warning
os.environ["LOG_LEVEL"] = "warning"

In [2]:
import ast
import pandas as pd
from services.opensearch_index import OpenSearchService
from tqdm import tqdm
import textwrap
from typing import List


from openai import AzureOpenAI

from utils.path_utils import ensure_dir

In [3]:
correct_qa_df = pd.read_csv(
    "../data/live_rag_questions/dmds_combined_questions_1000.tsv", sep="\t", index_col=0
)
# convert strings of lists to actual lists
correct_qa_df[["document_ids", "context"]] = correct_qa_df[
    ["document_ids", "context"]
].map(ast.literal_eval)
correct_qa_df


,question,answer,context,document_ids,question_factuality,question_premise,question_phrasing,question_linguistic_variation,question_multi-doc,user_expertise-categorization,generation_timestamp,question_length,answer_length,context_length
qid,,,,,,,,,,,,,,
1,exocrine pancreatic insufficiency symptoms cau...,Exocrine pancreatic insufficiency (EPI) is a c...,[Get inside Wall Street with StreetInsider Pre...,[<urn:uuid:3b710308-c2a0-45cc-a851-d7cf8689c7c...,factoid,with-premise,long-search-query,similar-to-document,three-doc,novice,2025-05-01T22:47:46.423955,8,65,1459
2,traditional kofte ingredients food poisoning r...,Traditional köfte is made with minced beef or ...,[True bastions of Turkish cuisine: there’s a f...,[<urn:uuid:001884ec-61a4-42f0-a44b-46bd41f5331...,open-ended,direct,short-search-query,similar-to-document,multi-aspect,novice,2025-05-01T22:47:46.423955,6,98,2742
3,I'm interested in environmental organizations ...,The Commission for Environmental Cooperation (...,[Richard A. Morgan announced as new Executive ...,[<urn:uuid:dd1b1a2e-de6c-4333-a242-62b476f0231a>],open-ended,with-premise,verbose-and-natural,similar-to-document,single-doc,novice,2025-05-01T22:47:46.423955,25,69,528
4,As a mental health professional concerned with...,The patient journey typically begins with an a...,[Psychiatric Emergency Services (PES)\nThe Psy...,[<urn:uuid:396e3028-f0a8-40ba-bd3c-bee87fa30fb...,open-ended,with-premise,verbose-and-natural,similar-to-document,three-doc,expert,2025-05-01T22:47:46.423955,43,137,3404
5,As a culinary historian exploring ancient past...,The early cultures had distinct approaches to ...,"[Pasta is a universally enjoyed food, and almo...",[<urn:uuid:239fecec-95de-40d2-b243-4d9ba9bd53c...,factoid,with-premise,verbose-and-natural,distant-from-document,three-doc,expert,2025-05-01T22:47:46.423955,33,112,3750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,My uncle keeps saying he doesn't have a gambli...,"Such a meeting is called an intervention, and ...",[Although people most commonly associate inter...,[<urn:uuid:3dec898b-52e0-40a5-8507-16f518617d22>],open-ended,with-premise,verbose-and-natural,distant-from-document,single-doc,novice,2025-05-02T11:45:30.494741,36,91,583
997,ways export modified 3d object surface design ...,"For three-dimensional problems, surface plots ...",[How to Reuse a Deformed Shape as a Geometry I...,[<urn:uuid:4a723482-c3db-427c-b1c8-144f6217f60b>],open-ended,direct,long-search-query,distant-from-document,single-doc,novice,2025-05-02T11:45:30.494741,11,56,1420
998,I'm researching global environmental threats -...,Both CO2 absorption and chemical contamination...,[A third of the planet's agricultural land is ...,[<urn:uuid:6846b2ea-5334-4991-8c7b-a862537d742...,factoid,with-premise,verbose-and-natural,distant-from-document,comparison,expert,2025-05-02T11:45:30.494741,24,79,2313


In [4]:
correct_qa_df = (
    correct_qa_df[["question", "context", "document_ids"]].explode("context").reset_index(drop=False)
)
correct_qa_df['document_ids'] = correct_qa_df['document_ids'].apply(set)
correct_qa_df

,qid,question,context,document_ids
0,1,exocrine pancreatic insufficiency symptoms cau...,Get inside Wall Street with StreetInsider Prem...,{<urn:uuid:9848238d-ca6c-4a85-8a99-55ac51736f1...
1,1,exocrine pancreatic insufficiency symptoms cau...,The Digestive Process: What Is the Role of You...,{<urn:uuid:9848238d-ca6c-4a85-8a99-55ac51736f1...
2,2,traditional kofte ingredients food poisoning r...,True bastions of Turkish cuisine: there’s a fe...,{<urn:uuid:50580576-4e8d-46c8-a3f0-c27de002303...
3,2,traditional kofte ingredients food poisoning r...,While grocery shopping for the Father’s Day we...,{<urn:uuid:50580576-4e8d-46c8-a3f0-c27de002303...
4,3,I'm interested in environmental organizations ...,Richard A. Morgan announced as new Executive D...,{<urn:uuid:dd1b1a2e-de6c-4333-a242-62b476f0231a>}
...,...,...,...,...
1564,998,I'm researching global environmental threats -...,A third of the planet's agricultural land is a...,{<urn:uuid:51ca6f21-f0f4-4343-8b14-a7f059f0fe6...
1565,998,I'm researching global environmental threats -...,What is Ocean Acidification?\nSince the beginn...,{<urn:uuid:51ca6f21-f0f4-4343-8b14-a7f059f0fe6...
1566,999,benefits drawbacks peer mediated instruction d...,This chapter provides a descriptive review of ...,{<urn:uuid:ab80ee62-79aa-4e8b-860d-247f948e6ad...
1567,999,benefits drawbacks peer mediated instruction d...,STRATEGY: Ensure school-level expulsion and su...,{<urn:uuid:ab80ee62-79aa-4e8b-860d-247f948e6ad...


In [5]:
correct_qa_df.iloc[0,3]

{'<urn:uuid:3b710308-c2a0-45cc-a851-d7cf8689c7c4>',
 '<urn:uuid:9848238d-ca6c-4a85-8a99-55ac51736f1c>'}

In [6]:
def run_search(
    queries: List[str],
    top_k: int = 100,
):
    # For each question, retrieve 100 documents from OpenSearch - takes 9 minutes
    service = OpenSearchService()

    res = []
    for query in tqdm(queries):
        search_res = service.query_opensearch(query=query, top_k=top_k)
        res.append(search_res)

    res_df = (
        pd.DataFrame(res)[["hits"]]
        .assign(question=queries)
        .explode("hits")
        .reset_index(drop=True)
    )
    res_df = res_df.assign(
        doc_text=res_df["hits"].apply(lambda x: x.source.text),
        doc_id=res_df["hits"].apply(lambda x: x.source.doc_id),
    ).drop(columns=["hits"])

    res_df.to_csv(
        f"../data/live_rag_questions/dmds_combined_questions_{len(queries)}_opensearch_{top_k}-results.tsv",
        sep="\t",
        index=False,
    )

    return res_df


# run_search(
#     queries=correct_qa_df["question"].unique(),
#     top_k=1000,
# )

res_df = pd.read_csv(
    "../data/live_rag_questions/dmds_combined_questions_1000_opensearch_1000-results.tsv",
    sep="\t",
)
res_df

,question,doc_text,doc_id
0,exocrine pancreatic insufficiency symptoms cau...,Exocrine pancreatic insufficiency (EPI) is cha...,<urn:uuid:a4c85ebf-b52e-4fa1-859d-eafa27d6a7da>
1,exocrine pancreatic insufficiency symptoms cau...,Biliary dyskinesia)\n- Pancreatic: Pancreatiti...,<urn:uuid:7e4d0cb7-233c-441c-95a8-da5fa32dfba3>
2,exocrine pancreatic insufficiency symptoms cau...,Different theories have been proposed why diab...,<urn:uuid:3f7da26e-b111-42b3-a34a-24329d8275a8>
3,exocrine pancreatic insufficiency symptoms cau...,Exocrine pancreatic insufficiency syndrome in ...,<urn:uuid:751de634-e494-428b-bf85-0569826de5cb>
4,exocrine pancreatic insufficiency symptoms cau...,The pancreas is a small pink organ located und...,<urn:uuid:d72602f5-aa76-410d-a9ce-970f470d2ec2>
...,...,...,...
999995,wga strike members representation negotiations,Posted BY: Justin O. Smith\nA railway strike h...,<urn:uuid:c7b3ba80-3ebc-420f-87b0-2d0a04e74bb0>
999996,wga strike members representation negotiations,"""These allegations are without merit, and this...",<urn:uuid:c5f468ba-d540-486f-ab8d-3ae4ec82bff0>
999997,wga strike members representation negotiations,"Moreover, this protocol details a robust and r...",<urn:uuid:450830e8-e063-4fc5-9b4f-995899ce138a>
999998,wga strike members representation negotiations,"Medicine, Issue 85, Transgenic mice, breast ca...",<urn:uuid:505d2f80-7355-4b6c-8627-c08a43728a0c>


In [7]:
def remove_relevant_docs_from_candidates(candidate_df, correct_df, nth=0):
    count = 0
    for row in candidate_df.itertuples():
        if row.doc_id in correct_df[correct_df["question"] == row.question]["document_ids"].values[0]:
            candidate_df.loc[row.Index, 'doc_id'] = None
            count += 1
    print(f"Number of reference documents: {count}")

    if count > 0:
        for row in candidate_df[candidate_df["doc_id"].isna()].itertuples():
            for _row in res_df[res_df["question"] == row.question].iloc[nth:].itertuples():
                if _row.doc_id in correct_df[correct_df["question"] == row.question]["document_ids"].values[0]:
                    continue
                else:
                    print(f"For question:\n {row.question},\n replacing {row.doc_id} with {_row.doc_id}")
                    print(f"Replacing {row.Index} with {_row.Index}")
                    candidate_df.loc[row.Index, 'doc_id'] = _row.doc_id
                    candidate_df.loc[row.Index, 'doc_text'] = _row.doc_text
                    print(f"Replacing:\n{row.doc_text}")
                    print(f"with:\n{_row.doc_text}")
                    count -= 1
                    break
    print(f"Number of reference documents in candidates: {count}")
    return candidate_df

# candidate_qa_df = res_df.groupby("question").nth(100).reset_index(drop=True)
# candidate_qa_df = remove_relevant_docs_from_candidates(candidate_qa_df, correct_qa_df)
# candidate_qa_df

In [8]:
def get_azure_openai_client():
    endpoint = os.getenv("ENDPOINT_URL", "https://relevance-judgments.openai.azure.com/")
    subscription_key = os.getenv("AZURE_OPENAI_API_KEY")

    # Initialize Azure OpenAI Service client with key-based authentication
    client = AzureOpenAI(
        azure_endpoint=endpoint,
        api_key=subscription_key,
        api_version="2025-01-01-preview",
    )
    return client


def prepare_chat_prompt(user_prompt, system_instructions=None):
    if system_instructions:
        chat_prompt = [
            {
                "role": "system",
                "content": [{"type": "text", "text": system_instructions}],
                "role": "user",
                "content": [{"type": "text", "text": user_prompt}],
            }
        ]
    else:
        chat_prompt = [
            {"role": "user", "content": [{"type": "text", "text": user_prompt}]}
        ]
    return chat_prompt


def get_chat_response(client, messages, model="gpt-4o"):
    # Generate the completion
    completion = client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=50,
        temperature=0,  # 0.7 is the default, 0 is deterministic
        top_p=1,  # 0.95 is the default
        frequency_penalty=0.5,
        stop=None,
        stream=False,
    )
    return completion

In [9]:
user_instructions_prompt = """Please read the question and the passage below and indicate how relevant the passage is to the query. 
Use the following scale: 

* 3 for perfectly relevant: The passage is dedicated to the query and contains the exact answer. 
* 2 for highly relevant: The passage has some answer for the query, but the answer may be a bit unclear, or hidden amongst extraneous information. 
* 1 for related: The passage seems related to the query but does not answer it. 
* 0 for irrelevant: The passage has nothing to do with the query. 

Question: 
{query} 

Passage: 
{passage} 

Indicate how relevant the passage is, using the scale above. 
Give only a number, do not give any explanation."""


In [10]:
def judge_qa_pairs(df, model, output_dir=None, iteration=0):
    if output_dir is None:
        _dir = ensure_dir(f"../data/live_rag_questions/azure_openai_responses/{model}")
    else:
        _dir = ensure_dir(output_dir)

    # Initialize the OpenAI client
    client = get_azure_openai_client()

    _df = df.copy()

    responses = []
    for row in tqdm(_df.itertuples(), total=len(_df)):
        query = row.question
        passage = row.doc_text

        # Prepare the chat prompt
        chat_prompt = prepare_chat_prompt(
            user_prompt=user_instructions_prompt.format(query=query, passage=passage)
        )
        try:
            # Get the chat response
            completion = get_chat_response(client, chat_prompt, model=model)
        except Exception as e:
            print(f"Error: {e}")
            print(f"Row: {row.Index}")
            print(f"Query: {query}")
            print(f"Passage: {passage}")
            with open(f"{_dir}/{row.Index}-{iteration}-{model}.error", "w") as f:
                f.write(str(e))
        else:
            # save the response to a json file
            with open(f"{_dir}/{row.Index}-{iteration}-{model}.json", "w") as f:
                f.write(completion.to_json())

        # Extract the response text
        response_text = None
        # cast the response to a dict
        try:
            completion = completion.to_dict()
        except Exception as e:
            print(f"Error: {e}")
            print(f"Row: {row.Index}")
            completion = {}
        choices = completion.get("choices")
        if choices:
            # Decode the response
            message = choices[0].get("message")
            if message:
                response_text = message.get("content")

        # Append the decoded response to the list
        responses.append(response_text)
    _df[f"{model}_response"] = responses
    _df.to_csv(
        f"../data/live_rag_questions/dmds_combined_questions_{len(_df)}_opensearch_results_azure_openai_{model}_responses-{iteration}.tsv",
        sep="\t",
        index=False,
    )
    return _df


# # examine the responses
# def examine_responses(df):
#     for idx, row in df[df["gpt-4o_response"].fillna("7").astype(int) > 0].iterrows():
#         print(f"Row: {idx}")
#         print(f"Query: {textwrap.fill(row.question, 100)}\n")
#         print(f"Passage: {textwrap.fill(row.doc_text, 120)}\n")
#         print(f"Response: {row['gpt-4o_response']}")
#         print("-" * 50)

In [ ]:
def select_negative_samples(
    res_df,
    correct_qa_df,
    max_iterations=10,
    start_document_rank=45,
    model="gpt-4.1-mini",
):
    negative_samples_df = pd.DataFrame()

    # get the first candidates
    n_questions = res_df["question"].nunique()
    _df = res_df.groupby("question").nth(start_document_rank).reset_index(drop=True)
    _df = remove_relevant_docs_from_candidates(
        _df, correct_qa_df, nth=start_document_rank
    )

    for i in range(1, max_iterations + 1):

        # judge the candidates
        _df = judge_qa_pairs(_df, model=model, iteration=i)
        _df[f"{model}_response"] = _df[f"{model}_response"].fillna("7").astype(int)

        print(
            f"Iteration {i}, judging results: {_df.value_counts(f'{model}_response')}"
        )

        negative_samples_df = pd.concat(
            [
                negative_samples_df,
                _df.loc[_df[f"{model}_response"] == 0],
            ],
            ignore_index=True,
        )
        print(f"Iteration {i}: {len(negative_samples_df)} negative samples")

        if len(negative_samples_df) < 10:
            return negative_samples_df, _df

        if len(negative_samples_df) >= n_questions:
            print(f"Found {len(negative_samples_df)} negative samples, stopping.")
            break

        next_candidates = _df.loc[_df[f"{model}_response"] > 0].copy()

        # set the next document rank to start from
        _nth = start_document_rank * (i + 1)

        next_candidates = (
            res_df.loc[res_df["question"].isin(next_candidates["question"])]
            .groupby("question")
            .nth(_nth)
            .reset_index(drop=True)
        )

        next_candidates = remove_relevant_docs_from_candidates(
            next_candidates, correct_qa_df, nth=_nth
        )

        _df = next_candidates.copy()
    else:
        print(f"Found {len(negative_samples_df)} negative samples, stopping.")
        print(f"Concatenating all last samples with the negative samples")
        negative_samples_df = pd.concat(
            [
                negative_samples_df,
                _df,
            ],
            ignore_index=True,
        )

    # save the negative samples
    negative_samples_df.to_csv(
        f"../data/live_rag_questions/negative_samples_{len(negative_samples_df)}-{model}.tsv",
        sep="\t",
        index=False,
    )
    return negative_samples_df


# negative_samples_df_1 = select_negative_samples(
#     res_df,
#     correct_qa_df,
#     max_iterations=15,
#     start_document_rank=65,
#     model="gpt-4.1-mini",
# )
negative_samples_df_1

Number of reference documents: 2
For question:
 what was role ra egyptian god,
 replacing None with <urn:uuid:930b2044-e9c1-42e3-9d73-38f7f52ca2c9>
Replacing 291 with 291066
Replacing:
The religious beliefs of the ancient Egyptians were the dominating influence in the development of their culture. The Egyptian faith was based on a collection of ancient myths, nature worship, and innumerable deities. In the most influential and famous of these myths a divine hierarchy is developed and the creation of the earth is explained.
According to the Egyptian account of creation, only the premordial waters existed at first. Then Ra, the sun, came out of an egg (a flower, in some versions) that appeared on the surface of the water. Ra brought forth four children, the gods Shu and Geb and the goddesses Tefnut and Nut. Shu and Tefnut became the atmosphere. They stood on Geb, who became the earth, and raised up Nut, who became the sky. Ra ruled over all. Geb and Nut later had two sons, Set and Osiris

 60%|██████    | 604/1000 [05:02<03:11,  2.06it/s]

Error: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': True, 'severity': 'medium'}, 'violence': {'filtered': True, 'severity': 'medium'}}}}}
Row: 603
Query: What abilities and companions distinguish the fourth horseman?
Passage: (Sahih Muslim 4457
At the massacre of Banu Qurayzah, Muhammad ordered his SS Jihadist Executioners known as the companions to behead all the Jewish men and in order to decide whi

100%|██████████| 1000/1000 [08:18<00:00,  2.01it/s]


Iteration 1, judging results: gpt-4.1-mini_response
1    438
0    413
2    133
3     15
7      1
Name: count, dtype: int64
Iteration 1: 413 negative samples
Number of reference documents: 3
For question:
 Which approach ensures higher workspace utilization: assigned seating or hot-desking?,
 replacing None with <urn:uuid:e75af4f4-d8b3-4c06-b5f2-d4c5165c8062>
Replacing 139 with 239131
Replacing:
So, now you know everything you need to know about hot desking!
with:
Now open! west elm WORKSPACE in Denver
Our west elm WORKSPACE showroom is now open! Come view the collection that's redefining the way you work. Call today...303.297.1100 for an appointment.
One system. Infinite applications. A small kit of parts, maximum flexibility, and timeless good looks allow you to create smart workspace applications for any work style.
Global Princeton Desks
Leave the work cubicle behind. Princeton is for a new generation that believes the office doesn’t have to be ‘the office’. It’s more than just a ne

...

100%|██████████| 64/64 [00:34<00:00,  1.83it/s]


Iteration 14, judging results: gpt-4.1-mini_response
1    41
2    14
0     7
3     2
Name: count, dtype: int64
Iteration 14: 943 negative samples
Number of reference documents: 0
Number of reference documents in candidates: 0


100%|██████████| 57/57 [00:30<00:00,  1.89it/s]

Iteration 15, judging results: gpt-4.1-mini_response
1    30
2    18
0     8
3     1
Name: count, dtype: int64
Iteration 15: 951 negative samples
Number of reference documents: 0
Number of reference documents in candidates: 0
Found 951 negative samples, stopping.
Concatenating all last samples with the negative samples


,question,doc_text,doc_id,gpt-4.1-mini_response
0,traditional kofte ingredients food poisoning r...,"THURSDAY, Jan. 8, 2015 (HealthDay News) -- The...",<urn:uuid:fbd1d404-24b9-48f0-8d7b-f233e6af2c0e>,0.0
1,I'm interested in environmental organizations ...,There are lots of organizations out there that...,<urn:uuid:9ffe53ea-c8c6-42e1-973e-7a09fdf84629>,0.0
2,Where was the Christmas message filmed?,We've just launched the MyHeritage Christmas c...,<urn:uuid:2526619d-b41f-4e63-bff4-73a7647fccbb>,0.0
3,I'm researching teacher training programs and ...,DOEE offers a suite of trainings that help Dis...,<urn:uuid:92f510ca-e614-4810-8a1c-f49832ebd5a0>,0.0
4,what happens first pregnancy doctor visit,"If you have any questions about contraception,...",<urn:uuid:77c2faa8-2fa7-4a14-b4e7-d51428233332>,0.0
...,...,...,...,...
946,body temperature comparison hypothermia frostb...,Fever Treatment : Fever is caused due to an in...,<urn:uuid:11177517-2317-4704-85b0-5cd3038f1333>,0.0
947,What should business managers look at when dec...,"However, Node.js, with its single-threaded eve...",<urn:uuid:81cd8d97-d176-4538-b836-182fec2c6818>,0.0
948,I'm managing a team and want to know - what ar...,Four steps to a sound electrical safety cultur...,<urn:uuid:2da7feda-8587-4745-8fff-948efaf89d2e>,0.0
949,elderly care options assistance types,Although the Alabama Student Grant Program is ...,<urn:uuid:8c914235-9822-45ea-a82e-436f0564ff1b>,0.0


In [ ]:
# negative_samples_df_2 = select_negative_samples(
#     res_df,
#     correct_qa_df,
#     max_iterations=15,
#     start_document_rank=65,
#     model="gpt-4o",
# )
negative_samples_df_2

Number of reference documents: 2
For question:
 what was role ra egyptian god,
 replacing None with <urn:uuid:930b2044-e9c1-42e3-9d73-38f7f52ca2c9>
Replacing 291 with 291066
Replacing:
The religious beliefs of the ancient Egyptians were the dominating influence in the development of their culture. The Egyptian faith was based on a collection of ancient myths, nature worship, and innumerable deities. In the most influential and famous of these myths a divine hierarchy is developed and the creation of the earth is explained.
According to the Egyptian account of creation, only the premordial waters existed at first. Then Ra, the sun, came out of an egg (a flower, in some versions) that appeared on the surface of the water. Ra brought forth four children, the gods Shu and Geb and the goddesses Tefnut and Nut. Shu and Tefnut became the atmosphere. They stood on Geb, who became the earth, and raised up Nut, who became the sky. Ra ruled over all. Geb and Nut later had two sons, Set and Osiris

 60%|██████    | 604/1000 [05:17<03:29,  1.89it/s]

Error: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': True, 'severity': 'medium'}, 'violence': {'filtered': True, 'severity': 'medium'}}}}}
Row: 603
Query: What abilities and companions distinguish the fourth horseman?
Passage: (Sahih Muslim 4457
At the massacre of Banu Qurayzah, Muhammad ordered his SS Jihadist Executioners known as the companions to behead all the Jewish men and in order to decide whi

100%|██████████| 1000/1000 [08:44<00:00,  1.91it/s]


Iteration 1, judging results: gpt-4o_response
0    637
1    278
2     61
3     23
7      1
Name: count, dtype: int64
Iteration 1: 637 negative samples
Number of reference documents: 2
For question:
 Which approach ensures higher workspace utilization: assigned seating or hot-desking?,
 replacing None with <urn:uuid:e75af4f4-d8b3-4c06-b5f2-d4c5165c8062>
Replacing 85 with 239131
Replacing:
So, now you know everything you need to know about hot desking!
with:
Now open! west elm WORKSPACE in Denver
Our west elm WORKSPACE showroom is now open! Come view the collection that's redefining the way you work. Call today...303.297.1100 for an appointment.
One system. Infinite applications. A small kit of parts, maximum flexibility, and timeless good looks allow you to create smart workspace applications for any work style.
Global Princeton Desks
Leave the work cubicle behind. Princeton is for a new generation that believes the office doesn’t have to be ‘the office’. It’s more than just a new look,

...

100%|██████████| 13/13 [00:06<00:00,  1.96it/s]


Iteration 14, judging results: gpt-4o_response
1    6
2    4
0    3
Name: count, dtype: int64
Iteration 14: 990 negative samples
Number of reference documents: 0
Number of reference documents in candidates: 0


100%|██████████| 10/10 [00:05<00:00,  1.73it/s]

Iteration 15, judging results: gpt-4o_response
1    6
3    2
0    1
2    1
Name: count, dtype: int64
Iteration 15: 991 negative samples
Number of reference documents: 0
Number of reference documents in candidates: 0
Found 991 negative samples, stopping.
Concatenating all last samples with the negative samples


,question,doc_text,doc_id,gpt-4o_response
0,exocrine pancreatic insufficiency symptoms cau...,About 94% of pancreatic cancers are classified...,<urn:uuid:2713c691-cd16-42dd-a9ee-55c819f37976>,0.0
1,traditional kofte ingredients food poisoning r...,"THURSDAY, Jan. 8, 2015 (HealthDay News) -- The...",<urn:uuid:fbd1d404-24b9-48f0-8d7b-f233e6af2c0e>,0.0
2,I'm interested in environmental organizations ...,There are lots of organizations out there that...,<urn:uuid:9ffe53ea-c8c6-42e1-973e-7a09fdf84629>,0.0
3,As a culinary historian exploring ancient past...,"ravioli and xiaolongbao\nseafood and tobiko, m...",<urn:uuid:8d0b135b-6322-40bc-8472-166a5be3c7ec>,0.0
4,Where was the Christmas message filmed?,We've just launched the MyHeritage Christmas c...,<urn:uuid:2526619d-b41f-4e63-bff4-73a7647fccbb>,0.0
...,...,...,...,...
986,inattentive adhd anxiety disorder physical sym...,Study: Methylphenidate is Effective Long-Term ...,<urn:uuid:ef1b5868-0d7a-4387-b864-7246ca1e945a>,0.0
987,ways start discussion cyberbullying teens,Free discussion questions are available for bo...,<urn:uuid:a9b9b431-6985-483a-89a2-94e8563f7e39>,0.0
988,preferred structures habitat fish locations,Monarch butterfly habitat focus of new environ...,<urn:uuid:12b54b80-b4d3-4140-aa70-bc02ebc45e7b>,0.0
989,ramsar wetlands environmental threats sustaina...,It will also assess how WFO could set up a pre...,<urn:uuid:acd7eb18-cdd8-4a88-bdcb-b465151f9322>,0.0


In [16]:
negative_samples_df_1.set_index("question").merge(
    negative_samples_df_2.set_index("question"),
    how="outer",
    left_index=True,
    right_index=True,
)

,doc_text_x,doc_id_x,gpt-4.1-mini_response,doc_text_y,doc_id_y,gpt-4o_response
question,,,,,,
Are social media and advocacy approaches similar between Hispanic and Native heritage celebrations?,Chiquita's top leader spoke of resilience and ...,<urn:uuid:1d70d876-017c-4a74-ad3c-37fb67c2f8de>,0.0,I embrace my roots and enlighten others about ...,<urn:uuid:5e13681f-078c-4c6e-a9f5-e0ae128d4c2f>,0.0
Are solar pumps good for small farms?,The city has been a leader in sustainable prac...,<urn:uuid:f602598b-6564-4e0f-bd4f-6f74e1697233>,0.0,Tag Archives: NJ\nNew Jersey is turning eyesor...,<urn:uuid:c2971b9d-d32b-4401-a812-1c81626d4869>,0.0
Are ticks or raccoons more dangerous for disease transmission?,NaN,NaN,NaN,Items 1-24 of 106\nFlea & Tick Treatments For ...,<urn:uuid:4ab73bd0-cce1-4645-b3ab-c213a29a36e5>,0.0
"As a GIS specialist interested in data modeling comparisons, I'm curious about the similarities between GIS data models and BIM - do both approaches focus on representing physical objects with detailed attributes throughout their lifecycle?",The certificate will be awarded when a student...,<urn:uuid:cf1a889c-b045-4d97-9e4e-bd09d8b60a2a>,0.0,The certificate will be awarded when a student...,<urn:uuid:cf1a889c-b045-4d97-9e4e-bd09d8b60a2a>,0.0
"As a breast cancer researcher studying viral links, how does MMTV transmission and treatment differ from BRCA-related cancers?",In findings that are fundamentally reshaping t...,<urn:uuid:d8c8d2ca-007c-477e-9ed2-9372e897a526>,0.0,From Our 2007 Archives\nBreast Cancer Genes Do...,<urn:uuid:a3034fa2-a7e0-417d-b27f-11cc89127857>,0.0
...,...,...,...,...,...,...
young criminal monitoring options effectiveness,Your doctor may want to test your liver functi...,<urn:uuid:32635e61-774c-426b-bd89-84c17457cdaa>,0.0,Your doctor may want to test your liver functi...,<urn:uuid:32635e61-774c-426b-bd89-84c17457cdaa>,0.0
young people mental health stress climate change comparison,"Community Mental Health Journal\nR.E., Essock,...",<urn:uuid:310edc27-e4a1-475b-a74a-48d86633db84>,0.0,From the Young Lawyers Division newsletter….\n...,<urn:uuid:662c009c-c85e-4e6e-ad4a-c803cdccd621>,0.0
youth group leader need quiet fun movement activities tips indoor spaces small area suggestions silent group games,Our mission is to provide youth a place where ...,<urn:uuid:0ee5dd30-c373-4bca-b139-5c41e981ebd0>,0.0,Our mission is to provide youth a place where ...,<urn:uuid:0ee5dd30-c373-4bca-b139-5c41e981ebd0>,0.0


In [21]:
from glob import glob

gpt41_judgments_files = glob(
    "../data/live_rag_questions/dmds_combined_questions_*_opensearch_results_azure_openai_gpt-4.1-mini_responses-*.tsv"
)
gpt41_judgments_df = pd.concat(
    [pd.read_csv(f, sep="\t") for f in gpt41_judgments_files],
    ignore_index=True,
)
gpt41_judgments_df

gpt4o_judgments_files = glob(
    "../data/live_rag_questions/dmds_combined_questions_*_opensearch_results_azure_openai_gpt-4o_responses-*.tsv"
)
gpt4o_judgments_df = pd.concat(
    [pd.read_csv(f, sep="\t") for f in gpt4o_judgments_files],
    ignore_index=True,
)
gpt4o_judgments_df

,question,doc_text,doc_id,gpt-4o_response
0,How can companies protect their intellectual p...,A technology industry organization says digita...,<urn:uuid:84538267-159d-4436-b306-62ec8f0f6a2f>,2.0
1,color space conversion between rgb cmyk differ...,This is a great online color tester and design...,<urn:uuid:91159e73-0b04-4022-9e25-dda1cfb288bf>,1.0
2,cbt vs cft techniques treating mental health,"I’m especially not ok with this, because as a ...",<urn:uuid:d2b904f8-ea05-4c25-a0d0-99078ae1ce33>,1.0
3,ways start discussion cyberbullying teens,Free discussion questions are available for bo...,<urn:uuid:a9b9b431-6985-483a-89a2-94e8563f7e39>,0.0
4,manufacturing process monitoring tools applica...,The Remsdaq SCADA systems division designs and...,<urn:uuid:889621be-3fa1-4465-b192-4c56ef0a831d>,1.0
...,...,...,...,...
1971,long term dental bridge 4 implants outcomes,All-on-4 Dental Implants Specialist in Anaheim...,<urn:uuid:8f2f502f-4d53-41f1-82d9-96d33e1d57f7>,1.0
1972,I'm comparing water conservation methods: buck...,: I am building a new home in U.K. and I would...,<urn:uuid:c1544db3-d4f1-4b98-aab2-4bd9c977e9bd>,1.0
1973,prevention techniques treatment options torn a...,"Receiving Jefferson Barracks, MO Sports Therap...",<urn:uuid:243391d3-7bd1-44bd-a1b4-9d0dbe57635d>,2.0
1974,reef tourism business environmental effects,"For example, in the center of the below photo,...",<urn:uuid:5de2164a-0948-424d-bd7f-7575f8d50471>,1.0


In [35]:
gpt41_judgments_df.groupby("question")["gpt-4.1-mini_response"].sum().sort_values(ascending=False)

question
How can team members help each other work better together?                                                                                                                                                                                    28.0
How can companies protect their intellectual property?                                                                                                                                                                                        27.0
How can I make my garden plants grow healthy?                                                                                                                                                                                                 27.0
tooth decay causes prevention benefits establishing dental home                                                                                                                                                                               26.0
intellectual proper

In [38]:
gpt4o_judgments_df.groupby("question")["gpt-4o_response"].sum().sort_values(ascending=False)

question
How can I make my garden plants grow healthy?                                                      29.0
How can team members help each other work better together?                                         29.0
manufacturing process monitoring tools applications                                                23.0
How can companies protect their intellectual property?                                             23.0
forest conservation measures extinction prevention biological diversity preservation approaches    22.0
                                                                                                   ... 
What defined Teddy Roosevelt's understanding of international relations?                            0.0
What defines a tree in graph theory terms?                                                          0.0
What did Padmasambhava teach about Vajrakīlaya meditation?                                          0.0
What different observations require a thorough sputum s

In [63]:
combined_judgments_df = gpt41_judgments_df.set_index(["question", "doc_text"]).merge(
    gpt4o_judgments_df.set_index(["question", "doc_text"]),
    left_index=True,
    right_index=True,
    how="outer",
)

_scores_sum = combined_judgments_df.fillna(1).sum(1, numeric_only=True).sort_values(
    ascending=False
)
not_relevant_pairs_df = _scores_sum.loc[_scores_sum < 2]
for i in not_relevant_pairs_df.index[0]:
    print(textwrap.fill(i, 100))
    print("-" * 50)
# write the not relevant pairs to a file
not_relevant_pairs_df.index.to_frame().to_json(
    "../data/live_rag_questions/not_relevant_pairs-1358.json",
    orient="records",
    lines=True,
)

stop motion animation how is it made
--------------------------------------------------
The sketch will start out similar to this link: [login to view URL] After which it will transform to
a full design with my logos. I already have my logos designed. I am running a sport motion video
analyze project and need assistance for the initial video data analyze. I have around 70 GByte of
high speed video recordings that need to be analyzed so a relationship between time steps and the
motion is made (mm motion in upward y direction), and it MUST be made by using the analyzing tool
Tracker 5.0 (see descriptio... Hello Freelancer, I need a logo for my website. Its name Motion
Bucket. Send your demo design with your proper budget. I'll hire those who can also create my full
brand identity like business card, cover design etc WE HAVE OFFICE LAN WITH ONE SERVER AND 15 PCS
MAINLY USED FOR TALLY AND REGULAR OFFICE APPLICATIONS. IT IS OBSERVED THAT THE SERVER IS NOT FULLY
UTILISED BUT THE PCS ARE HANG